In [24]:
%pip install -Uq boto3 anthropic

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -r requirements.txt

In [1]:
from dotenv import load_dotenv
from utils.visualize import visualize
from typing import List, Dict,Optional
load_dotenv()

# MODEL= 'global.anthropic.claude-opus-4-5-20251101-v1:0'
MODEL = 'global.anthropic.claude-sonnet-4-5-20250929-v1:0'

viz = visualize(auto_show=True)

In [3]:
import json

from utils.team_expense_api import get_custom_budget, get_expenses, get_team_members

from anthropic import AnthropicBedrock

client = AnthropicBedrock()

message = client.messages.create(
    model=MODEL,
    max_tokens=256,
    messages=[{"role": "user", "content": "Hello, world"}]
)
print(message.content)

[TextBlock(citations=None, text='Hello! How can I help you today?', type='text')]


In [ ]:
DIRECT_TOOL_CONFIGS = [
    {
        "name": "get_weather",
        "description": "Get current weather information for a city. This tool provides real-time weather data including temperature, conditions, humidity, and wind speed. Use this when you need to check weather conditions for travel planning or general information.",
        "input_schema": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city name to get weather for (e.g., 'Beijing', 'New York', 'London')",
                },
                "units": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature units. Default is 'celsius'.",
                    "default": "celsius"
                }
            },
            "required": ["city"],
        },
    },
]
# Tool definitions for the team expense API
tools = [
    {
        "name": "get_team_members",
        "description": 'Returns a list of team members for a given department. Each team member includes their ID, name, role, level (junior, mid, senior, staff, principal), and contact information. Use this to get a list of people whose expenses you want to analyze. Available departments are: engineering, sales, and marketing.\n\nRETURN FORMAT: Returns a JSON string containing an ARRAY of team member objects (not wrapped in an outer object). Parse with json.loads() to get a list. Example: [{"id": "ENG001", "name": "Alice", ...}, {"id": "ENG002", ...}]',
        "input_schema": {
            "type": "object",
            "properties": {
                "department": {
                    "type": "string",
                    "description": "The department name. Case-insensitive.",
                }
            },
            "required": ["department"],
        },
        "input_examples": [
            {"department": "engineering"},
            {"department": "sales"},
            {"department": "marketing"},
        ],
    },
    {
        "name": "get_expenses",
        "description": "Returns all expense line items for a given employee in a specific quarter. Each expense includes extensive metadata: date, category, description, amount (in USD), currency, status (approved, pending, rejected), receipt URL, approval chain, merchant name and location, payment method, and project codes. An employee may have 20-50+ expense line items per quarter, and each line item contains substantial metadata for audit and compliance purposes. Categories include: 'travel' (flights, trains, rental cars, taxis, parking), 'lodging' (hotels, airbnb), 'meals', 'software', 'equipment', 'conference', 'office', and 'internet'. IMPORTANT: Only expenses with status='approved' should be counted toward budget limits.\n\nRETURN FORMAT: Returns a JSON string containing an ARRAY of expense objects (not wrapped in an outer object with an 'expenses' key). Parse with json.loads() to get a list directly. Example: [{\"expense_id\": \"ENG001_Q3_001\", \"amount\": 1250.50, \"category\": \"travel\", ...}, {...}]",
        "input_schema": {
            "type": "object",
            "properties": {
                "employee_id": {
                    "type": "string",
                    "description": "The unique employee identifier",
                },
                "quarter": {
                    "type": "string",
                    "description": "Quarter identifier: 'Q1', 'Q2', 'Q3', or 'Q4'",
                },
            },
            "required": ["employee_id", "quarter"],
        },
        "input_examples": [
            {"employee_id": "ENG001", "quarter": "Q3"},
            {"employee_id": "SAL002", "quarter": "Q1"},
            {"employee_id": "MKT001", "quarter": "Q4"},
        ],
    },
    {
        "name": "get_custom_budget",
        "description": 'Get the custom quarterly travel budget for a specific employee. Most employees have a standard $5,000 quarterly travel budget. However, some employees have custom budget exceptions based on their role requirements. This function checks if a specific employee has a custom budget assigned.\n\nRETURN FORMAT: Returns a JSON string containing a SINGLE OBJECT (not an array). Parse with json.loads() to get a dict. Example: {"user_id": "ENG001", "has_custom_budget": false, "travel_budget": 5000, "reason": "Standard", "currency": "USD"}',
        "input_schema": {
            "type": "object",
            "properties": {
                "user_id": {
                    "type": "string",
                    "description": "The unique employee identifier",
                }
            },
            "required": ["user_id"],
        },
        "input_examples": [
            {"user_id": "ENG001"},
            {"user_id": "SAL002"},
            {"user_id": "MKT001"},
        ],
    },
]

tool_functions = {
    "get_team_members": get_team_members,
    "get_expenses": get_expenses,
    "get_custom_budget": get_custom_budget,
}

### Traditional Tool Calling (Baseline)
In this first example, we'll use traditional tool calling to establish our baseline.

We'll call the messages.create API with our initial query. When the model stops with a tool_use reason, we will execute the tool as requested, and then add the output from the tool to the messages and call the model again.

In [5]:
import time

from anthropic.types import TextBlock, ToolUseBlock
from anthropic.types.beta import (
    BetaMessageParam as MessageParam,
)
from anthropic.types.beta import (
    BetaTextBlock,
    BetaToolUseBlock,
)


In [5]:

messages: list[MessageParam] = []


def run_agent_without_ptc(user_message):
    """Run agent using traditional tool calling"""
    messages.append({"role": "user", "content": user_message})
    total_tokens = 0
    start_time = time.time()
    api_counter = 0

    while True:
        response = client.beta.messages.create(
            model=MODEL,
            max_tokens=8000,
            tools=tools,
            messages=messages,
            # betas=["tool-examples-2025-10-29"],
        )

        api_counter += 1

        # Track token usage
        total_tokens += response.usage.input_tokens + response.usage.output_tokens
        viz.capture(response)
        if response.stop_reason == "end_turn":
            # Extract the first text block from the response
            final_response = next(
                (
                    block.text
                    for block in response.content
                    if isinstance(block, (BetaTextBlock, TextBlock))
                ),
                None,
            )
            elapsed_time = time.time() - start_time
            return final_response, messages, total_tokens, elapsed_time, api_counter

        # Process tool calls
        if response.stop_reason == "tool_use":
            # First, add the assistant's response to messages
            messages.append({"role": "assistant", "content": response.content})

            # Collect all tool results
            tool_results = []

            for block in response.content:
                if isinstance(block, (BetaToolUseBlock, ToolUseBlock)):
                    tool_name = block.name
                    tool_input = block.input
                    tool_use_id = block.id

                    result = tool_functions[tool_name](**tool_input)

                    content = str(result)

                    tool_result = {
                        "type": "tool_result",
                        "tool_use_id": tool_use_id,
                        "content": content,
                    }
                    tool_results.append(tool_result)

            # Append all tool results at once after collecting them
            messages.append({"role": "user", "content": tool_results})

        else:
            print(f"\nUnexpected stop reason: {response.stop_reason}")
            elapsed_time = time.time() - start_time

            final_response = next(
                (
                    block.text
                    for block in response.content
                    if isinstance(block, (BetaTextBlock, TextBlock))
                ),
                f"Stopped with reason: {response.stop_reason}",
            )
            return final_response, messages, total_tokens, elapsed_time, api_counter

In [6]:
query = "Which engineering team members exceeded their Q3 travel budget? Standard quarterly travel budget is $5,000. However, some employees have custom budget limits. For anyone who exceeded the $5,000 standard budget, check if they have a custom budget exception. If they do, use that custom limit instead to determine if they truly exceeded their budget."


In [7]:
# Run the agent
result, conversation, total_tokens, elapsed_time, api_count_without_ptc = run_agent_without_ptc(
    query
)

print(f"Result: {result}")
print(f"API calls made: {api_count_without_ptc}")
print(f"Total tokens used: {total_tokens:,}")
print(f"Total time taken: {elapsed_time:.2f}s")

╭────────────────────────────────────────────── Claude API Response ──────────────────────────────────────────────╮
│ Claude Message (assistant) │ tokens: 1,333 in • 85 out • 1,418 total                                            │
│ ├── Model: claude-sonnet-4-5-20250929                                                                           │
│ ├── Stop Reason: tool_use                                                                                       │
│ └── Content (2 blocks)                                                                                          │
│     ├── Block 1                                                                                                 │
│     │   └── Text                                                                                                │
│     │       └── I'll help you identify which engineering team members exceeded their Q3 travel budget. Let me   │
│     │           start by getting the list of engineering team members.                                          │
│     └── Block 2                                                                                                 │
│         └── Tool Use: get_team_members                                                                          │
│             ├── ID: toolu_bdrk_016gkzeKtHsCKtco38jkp6Mw                                                         │
│             └── Input:                                                                                          │
│                 └── {                                                                                           │
│                       "department": "engineering"                                                               │
│                     }                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── Claude API Response ───────────────────────────────╮
│ Claude Message (assistant) │ tokens: 1,947 in • 497 out • 2,444 total            │
│ ├── Model: claude-sonnet-4-5-20250929                                            │
│ ├── Stop Reason: tool_use                                                        │
│ └── Content (9 blocks)                                                           │
│     ├── Block 1                                                                  │
│     │   └── Text                                                                 │
│     │       └── Now let me get the Q3 expenses for all engineering team members. │
│     ├── Block 2                                                                  │
│     │   └── Tool Use: get_expenses                                               │
│     │       ├── ID: toolu_bdrk_01LLACS4jT3S6h8WHyFRtuCy                          │
│     │       └── Input:                                                           │
│     │           └── {                                                            │
│     │                 "employee_id": "ENG001",                                   │
│     │                 "quarter": "Q3"                                            │
│     │               }                                                            │
│     ├── Block 3                                                                  │
│     │   └── Tool Use: get_expenses                                               │
│     │       ├── ID: toolu_bdrk_01DMvdCs8XYYR2Mw9T48nR7i                          │
│     │       └── Input:                                                           │
│     │           └── {                                                            │
│     │                 "employee_id": "ENG002",                                   │
│     │                 "quarter": "Q3"                                            │
│     │               }                                                            │
│     ├── Block 4                                                                  │
│     │   └── Tool Use: get_expenses                                               │
│     │       ├── ID: toolu_bdrk_014r9vA7R7NBK1hzyD1kKEvC                          │
│     │       └── Input:                                                           │
│     │           └── {                                                            │
│     │                 "employee_id": "ENG003",                                   │
│     │                 "quarter": "Q3"                                            │
│     │               }                                                            │
│     ├── Block 5                                                                  │
│     │   └── Tool Use: get_expenses                                               │
│     │       ├── ID: toolu_bdrk_017YJHYvX48f4zRx5MBo1e7v                          │
│     │       └── Input:                                                           │
│     │           └── {                                                            │
│     │                 "employee_id": "ENG004",                                   │
│     │                 "quarter": "Q3"                                            │
│     │               }                                                            │
│     ├── Block 6                                                                  │
│     │   └── Tool Use: get_expenses                                               │
│     │       ├── ID: toolu_bdrk_019vtjLSSS6dM47oo2UCN2HP                          │
│     │       └── Input:                                                           │
│     │           └── {                                                            │
│     │                 "employee_id": "ENG005",                                   │
│     │                 "quarter": "Q3"                                            │
│    

╭────────────────────────────────────────────── Claude API Response ──────────────────────────────────────────────╮
│ Claude Message (assistant) │ tokens: 65,154 in • 396 out • 65,550 total                                         │
│ ├── Model: claude-sonnet-4-5-20250929                                                                           │
│ ├── Stop Reason: tool_use                                                                                       │
│ └── Content (9 blocks)                                                                                          │
│     ├── Block 1                                                                                                 │
│     │   └── Text                                                                                                │
│     │       └── Now let me calculate the travel expenses for each team member and check who exceeded their      │
│     │           budget. I'll need to sum up only the approved travel-related expenses (travel and lodging       │
│     │           categories with status="approved").                                                             │
│     ├── Block 2                                                                                                 │
│     │   └── Tool Use: get_custom_budget                                                                         │
│     │       ├── ID: toolu_bdrk_01RPAfFmDLQ7DrfxY7nsrJ2E                                                         │
│     │       └── Input:                                                                                          │
│     │           └── {                                                                                           │
│     │                 "user_id": "ENG001"                                                                       │
│     │               }                                                                                           │
│     ├── Block 3                                                                                                 │
│     │   └── Tool Use: get_custom_budget                                                                         │
│     │       ├── ID: toolu_bdrk_019ux4iT1Si9ovWBxw7hxxTy                                                         │
│     │       └── Input:                                                                                          │
│     │           └── {                                                                                           │
│     │                 "user_id": "ENG002"                                                                       │
│     │               }                                                                                           │
│     ├── Block 4                                                                                                 │
│     │   └── Tool Use: get_custom_budget                                                                         │
│     │       ├── ID: toolu_bdrk_01FywFPZZjTTVxYRwP7UUFEL                                                         │
│     │       └── Input:                                                                                          │
│     │           └── {                                                                                           │
│     │                 "user_id": "ENG003"                                                                       │
│     │               }                                                                                           │
│     ├── Block 5                                                                                                 │
│     │   └── Tool Use: get_custom_budget                                                                         │
│     │       ├── ID: toolu_bdrk_01VkVNS7bEJhznjp7R8XLyke                                                         │
│     │       └── Input:                                

╭────────────────────────────────────────────── Claude API Response ──────────────────────────────────────────────╮
│ Claude Message (assistant) │ tokens: 66,218 in • 1,593 out • 67,811 total                                       │
│ ├── Model: claude-sonnet-4-5-20250929                                                                           │
│ ├── Stop Reason: end_turn                                                                                       │
│ └── Content (1 blocks)                                                                                          │
│     └── Block 1                                                                                                 │
│         └── Text                                                                                                │
│             └── Now let me analyze the data. I need to calculate approved travel expenses (categories: 'travel' │
│                 and 'lodging' with status='approved') for each engineer:                                        │
│                                                                                                                 │
│                 **ENG001 (Alice Chen) - Standard Budget: $5,000**                                               │
│                 - Travel: $1,283.95 + $196.76 + $186.75 + $1,017.20 + $28.78 + $1,060.97 + $1,051.26 + $15.96 + │
│                 $175.48 + $1,492.55 = $6,509.66                                                                 │
│                 - Lodging: $1,621.08 + $1,166.78 = $2,787.86                                                    │
│                 - **Total: $9,297.52** ✅ EXCEEDED $5,000                                                       │
│                                                                                                                 │
│                 **ENG002 (Bob Martinez) - Custom Budget: $8,000**                                               │
│                 - Travel: $710.22 + $24.55 + $1,216.45 + $11.18 + $1,128.86 + $178.26 + $1,167.89 + $1,276.95 + │
│                 $1,484.08 + $1,435.73 = $10,634.17                                                              │
│                 - Lodging: $1,494.16 + $705.56 = $2,199.72                                                      │
│                 - **Total: $12,833.89** ✅ EXCEEDED $8,000 custom budget                                        │
│                                                                                                                 │
│                 **ENG003 (Carol White) - Standard Budget: $5,000**                                              │
│                 - Travel: $35.74 + $1,150.98 + $1,429.90 + $192.77 + $12.77 + $156.78 + $163.89 + $1,352.47 +   │
│                 $1,452.83 + $949.18 = $6,897.31                                                                 │
│                 - Lodging: $1,512.23 + $808.68 + $1,273.64 = $3,594.55                                          │
│                 - **Total: $10,491.86** ✅ EXCEEDED $5,000                                                      │
│                                                                                                                 │
│                 **ENG004 (David Kim) - Cu                                                                       │
│                 ... (truncated)                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Result: Now let me analyze the data. I need to calculate approved travel expenses (categories: 'travel' and 'lodging' with status='approved') for each engineer:

**ENG001 (Alice Chen) - Standard Budget: $5,000**
- Travel: $1,283.95 + $196.76 + $186.75 + $1,017.20 + $28.78 + $1,060.97 + $1,051.26 + $15.96 + $175.48 + $1,492.55 = $6,509.66
- Lodging: $1,621.08 + $1,166.78 = $2,787.86
- **Total: $9,297.52** ✅ EXCEEDED $5,000

**ENG002 (Bob Martinez) - Custom Budget: $8,000**
- Travel: $710.22 + $24.55 + $1,216.45 + $11.18 + $1,128.86 + $178.26 + $1,167.89 + $1,276.95 + $1,484.08 + $1,435.73 = $10,634.17
- Lodging: $1,494.16 + $705.56 = $2,199.72
- **Total: $12,833.89** ✅ EXCEEDED $8,000 custom budget

**ENG003 (Carol White) - Standard Budget: $5,000**
- Travel: $35.74 + $1,150.98 + $1,429.90 + $192.77 + $12.77 + $156.78 + $163.89 + $1,352.47 + $1,452.83 + $949.18 = $6,897.31
- Lodging: $1,512.23 + $808.68 + $1,273.64 = $3,594.55
- **Total: $10,491.86** ✅ EXCEEDED $5,000

**ENG004 (David K

# Customized PTC

In [24]:
!python examples/bedrock_docker_agent_example.py -v

2025-12-29 10:53:25,178 - asyncio - selector_events.py:64 - DEBUG - Using selector: EpollSelector
Bedrock Docker Sandbox Agent Demo
2025-12-29 10:53:25,178 - __main__ - bedrock_docker_agent_example.py:324 - INFO - Registered tool: get_team_members
2025-12-29 10:53:25,178 - __main__ - bedrock_docker_agent_example.py:324 - INFO - Registered tool: get_expenses
2025-12-29 10:53:25,178 - __main__ - bedrock_docker_agent_example.py:324 - INFO - Registered tool: get_custom_budget
2025-12-29 10:53:25,178 - __main__ - bedrock_docker_agent_example.py:342 - INFO - Registered direct tool: get_weather
2025-12-29 10:53:25,178 - __main__ - bedrock_docker_agent_example.py:302 - INFO - Initialized BedrockDockerSandboxAgent with model: global.anthropic.claude-sonnet-4-5-20250929-v1:0
2025-12-29 10:53:25,178 - __main__ - bedrock_docker_agent_example.py:346 - INFO - Docker sandbox agent ready

Docker Sandbox Agent created with 4 tools:
  - get_team_members (code_execution)
  - get_expenses (code_execution)

2025-12-29 10:53:25,587 - __main__ - bedrock_docker_agent_example.py:382 - INFO - AnthropicBedrock client initialized
╭──────────────────────────── Claude API Response ─────────────────────────────╮
│ Claude Message (assistant) │ tokens: 4,646 in • 1,394 out • 6,040 total      │
│ ├── Model: claude-sonnet-4-5-20250929                                        │
│ ├── Stop Reason: tool_use                                                    │
│ └── Content (2 blocks)                                                       │
│     ├── Block 1                                                              │
│     │   └── Text                                                             │
│     │       └── I'll help you identify which engineering team members        │
│     │           exceeded their Q3 travel budget. Let me fetch the data and   │
│     │           analyze it, checking for custom budgets where applicable.    │
│     └── Block 2                                                       

## Test for Antropic API Proxy

In [ ]:
import copy

ptc_tools = copy.deepcopy(tools)
for tool in ptc_tools:
    tool["allowed_callers"] = ["code_execution_20250825"]  # type: ignore


# Add the code execution tool
ptc_tools.append(
    {
        "type": "code_execution_20250825",  # type: ignore
        "name": "code_execution",
    }
)

In [22]:

def get_weather(city: str, units: str = "celsius") -> str:
    """
    Mock weather API - returns simulated weather data for a city.
    This is a DIRECT CALL tool - Claude calls it directly, not via code execution.
    """
    import random

    # Mock weather data for different cities
    weather_data = {
        "beijing": {"temp_c": 15, "condition": "Partly Cloudy", "humidity": 45, "wind_kph": 12},
        "shanghai": {"temp_c": 22, "condition": "Sunny", "humidity": 60, "wind_kph": 8},
        "new york": {"temp_c": 18, "condition": "Cloudy", "humidity": 55, "wind_kph": 15},
        "london": {"temp_c": 12, "condition": "Rainy", "humidity": 80, "wind_kph": 20},
        "tokyo": {"temp_c": 20, "condition": "Clear", "humidity": 50, "wind_kph": 10},
        "paris": {"temp_c": 14, "condition": "Overcast", "humidity": 65, "wind_kph": 18},
        "sydney": {"temp_c": 25, "condition": "Sunny", "humidity": 40, "wind_kph": 5},
        "san francisco": {"temp_c": 16, "condition": "Foggy", "humidity": 75, "wind_kph": 22},
    }

    city_lower = city.lower().strip()

    if city_lower in weather_data:
        data = weather_data[city_lower]
    else:
        # Generate random weather for unknown cities
        data = {
            "temp_c": random.randint(5, 35),
            "condition": random.choice(["Sunny", "Cloudy", "Rainy", "Clear", "Windy"]),
            "humidity": random.randint(30, 90),
            "wind_kph": random.randint(5, 30)
        }

    # Convert temperature if needed
    if units.lower() == "fahrenheit":
        temp = data["temp_c"] * 9/5 + 32
        temp_unit = "°F"
    else:
        temp = data["temp_c"]
        temp_unit = "°C"

    result = {
        "city": city.title(),
        "temperature": f"{temp:.1f}{temp_unit}",
        "condition": data["condition"],
        "humidity": f"{data['humidity']}%",
        "wind": f"{data['wind_kph']} km/h",
        "units": units.lower()
    }

    return json.dumps(result, ensure_ascii=False)


tool_functions['get_weather'] = get_weather

In [ ]:
import anthropic
messages = []


anthropic_client = anthropic.Anthropic(api_key='sk-22b986366e084cafae975331ae994e8a',
                                       base_url='http://127.0.0.1:8000')

message = anthropic_client.beta.messages.create(
    # model="qwen.qwen3-coder-480b-a35b-v1:0",
    # model="global.anthropic.claude-opus-4-5-20251101-v1:0",
    model = 'global.anthropic.claude-sonnet-4-5-20250929-v1:0',
    # model = 'minimax.minimax-m2',
    system="Your are a helpful agent",
    max_tokens=8000,
    messages=[
        {"role": "user", "content": "你好，Claude！"}
    ]
)

print(message.content)

[BetaTextBlock(citations=None, text='你好！很高兴见到你！有什么我可以帮助你的吗？', type='text', cache_control=None)]


In [35]:


def run_agent_with_ptc(user_message,model_id):
    """Run agent using PTC"""
    messages = []
    messages.append({"role": "user", "content": user_message})
    total_tokens = 0
    start_time = time.time()
    container_id = None
    api_counter = 0

    while True:
        # Build request with PTC beta headers
        request_params = {
            "model":model_id,
            "max_tokens": 8000,
            "tools": ptc_tools+DIRECT_TOOL_CONFIGS,
            "messages": messages,
        }

        response = anthropic_client.beta.messages.create(
            **request_params,
            system="Your are a helpful agent",
            thinking={
                "type": "enabled",
                "budget_tokens": 2048
            },
            betas=[
                "advanced-tool-use-2025-11-20",
                'interleaved-thinking-2025-05-14'
            ],
            extra_body={"container": container_id} if container_id else None,
        )
        viz.capture(response)
        api_counter += 1

        # Track container for stateful execution
        if hasattr(response, "container") and response.container:
            container_id = response.container.id
            print(f"\n[Container] ID: {container_id}")
            if hasattr(response.container, "expires_at"):
                # If the container has expired, we would need to restart our workflow. In our case, it completes before expiration.
                print(f"[Container] Expires at: {response.container.expires_at}")

        # Track token usage
        total_tokens += response.usage.input_tokens + response.usage.output_tokens

        if response.stop_reason == "end_turn":
            # Extract the first text block from the response
            final_response = next(
                (block.text for block in response.content if isinstance(block, BetaTextBlock)),
                None,
            )
            elapsed_time = time.time() - start_time
            return final_response, messages, total_tokens, elapsed_time, api_counter

        # As before, we process tool calls
        if response.stop_reason == "tool_use":
            # First, add the assistant's response to messages
            messages.append({"role": "assistant", "content": response.content})

            # Collect all tool results
            tool_results = []

            for block in response.content:
                if isinstance(block, BetaToolUseBlock):
                    tool_name = block.name
                    tool_input = block.input
                    tool_use_id = block.id

                    # We can use caller type to understand how the tool was invoked
                    caller_type = block.caller.type  # type: ignore
                    if caller_type == "code_execution_20250825":
                        print(f"[PTC] Tool called from code execution environment: {tool_name}")
        
                    elif caller_type == "direct":
                        print(f"[Direct] Tool called by model: {tool_name}")

                    result = tool_functions[tool_name](**tool_input)

                    # Format result as proper content for the API
                    if isinstance(result, list) and result and isinstance(result[0], str):
                        content = "\n".join(result)
                    elif isinstance(result, (dict, list)):
                        content = json.dumps(result)
                    else:
                        content = str(result)

                    tool_results.append(
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use_id,
                            "content": content,
                        }
                    )

            messages.append({"role": "user", "content": tool_results})

        else:
            print(f"\nUnexpected stop reason: {response.stop_reason}")
            elapsed_time = time.time() - start_time

            final_response = next(
                (block.text for block in response.content if isinstance(block, BetaTextBlock)),
                f"Stopped with reason: {response.stop_reason}",
            )
            return final_response, messages, total_tokens, elapsed_time, api_counter

In [36]:
# Run the PTC agent
# model_id = 'minimax.minimax-m2'
# model_id = 'qwen.qwen3-coder-480b-a35b-v1:0'
# model_id="global.anthropic.claude-opus-4-5-20251101-v1:0"
model_id = 'global.anthropic.claude-sonnet-4-5-20250929-v1:0'
query = "Which engineering team members exceeded their Q3 travel budget? Standard quarterly travel budget is $5,000. However, some employees have custom budget limits. For anyone who exceeded the $5,000 standard budget, check if they have a custom budget exception. If they do, use that custom limit instead to determine if they truly exceeded their budget."
# query = "what is the weather of beijing?"
result_ptc, conversation_ptc, total_tokens_ptc, elapsed_time_ptc, api_count_with_ptc = (
    run_agent_with_ptc(query,model_id)
)

print(f"Result: {result_ptc}")
print(f"API calls made: {api_count_with_ptc}")
print(f"Total tokens used: {total_tokens_ptc:,}")
print(f"Total time taken: {elapsed_time_ptc:.2f}s")

╭────────────────────────────────────────────── Claude API Response ──────────────────────────────────────────────╮
│ Claude Message (assistant) │ tokens: 5,013 in • 1,568 out • 6,581 total                                         │
│ ├── Model: global.anthropic.claude-sonnet-4-5-20250929-v1:0                                                     │
│ ├── Stop Reason: tool_use                                                                                       │
│ └── Content (3 blocks)                                                                                          │
│     ├── Block 1                                                                                                 │
│     │   └── Text                                                                                                │
│     │       └── I'll help you identify which engineering team members exceeded their Q3 travel budget. Let me   │
│     │           gather the data and analyze it.                                                                 │
│     ├── Block 2                                                                                                 │
│     │   └── Server Tool Use                                                                                     │
│     │       ├── ID: srvtoolu_7bd6844a5b1d                                                                       │
│     │       └── Code:                                                                                           │
│     │           └──    1                                                                                        │
│     │                  2 import json                                                                            │
│     │                  3 import asyncio                                                                         │
│     │                  4                                                                                        │
│     │                  5 # Step 1: Get all engineering team members                                             │
│     │                  6 team_data = await get_team_members(department="engineering")                           │
│     │                  7 team_members = json.loads(team_data)                                                   │
│     │                  8 print(f"Found {len(team_members)} engineering team members\n")                         │
│     │                  9                                                                                        │
│     │                 10 # Step 2: Get Q3 expenses for all team members in parallel                             │
│     │                 11 expense_tasks = [get_expenses(employee_id=member['id'], quarter='Q3') for member in te │
│     │                 12 expense_results = await asyncio.gather(*expense_tasks)                                 │
│     │                 13                                                                                        │
│     │                 14 # Step 3: Calculate travel expenses for each member (only approved, travel + lodging)  │
│     │                 15 travel_expenses = {}                                                                   │
│     │                 16 for member, expenses_json in zip(team_members, expense_results):                       │
│     │                 17     expenses = json.loads(expenses_json)                                               │
│     │                 18                                                                                        │
│     │                 19     # Sum approved travel and lodging expenses                                         │
│     │                 20     total_travel = sum(                                                                │
│     │                 21         exp['amount'] for exp in expenses                                              │
│     │                 22         if exp['status'] == '


[Container] ID: container_55a4b5c74f93
[Container] Expires at: 2025-12-29 13:52:54.239663
[PTC] Tool called from code execution environment: get_team_members


╭──────────────────── Claude API Response ────────────────────╮
│ Claude Message (assistant) │ tokens: 0 in • 0 out • 0 total │
│ ├── Model: global.anthropic.claude-sonnet-4-5-20250929-v1:0 │
│ ├── Stop Reason: tool_use                                   │
│ └── Content (8 blocks)                                      │
│     ├── Block 1                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_b566f58a-319                      │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG001",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 2                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_1b6d8f49-779                      │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG002",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 3                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_7171cbbd-c87                      │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG003",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 4                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_cc1ebe4e-8d0                      │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG004",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 5                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_8bd10163-11b                      │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG005",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 6                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_8a860905-3b9                      │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG006",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 7                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_c3cd55ff-159                      │
│     │       ├── Caller: code e


[Container] ID: container_55a4b5c74f93
[Container] Expires at: 2025-12-29 13:52:54.239663
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses


╭──────────────────── Claude API Response ────────────────────╮
│ Claude Message (assistant) │ tokens: 0 in • 0 out • 0 total │
│ ├── Model: global.anthropic.claude-sonnet-4-5-20250929-v1:0 │
│ ├── Stop Reason: tool_use                                   │
│ └── Content (4 blocks)                                      │
│     ├── Block 1                                             │
│     │   └── Tool Use: get_custom_budget                     │
│     │       ├── ID: toolu_605679d6-888                      │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "user_id": "ENG002"                   │
│     │               }                                       │
│     ├── Block 2                                             │
│     │   └── Tool Use: get_custom_budget                     │
│     │       ├── ID: toolu_6e30256e-8d2                      │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "user_id": "ENG003"                   │
│     │               }                                       │
│     ├── Block 3                                             │
│     │   └── Tool Use: get_custom_budget                     │
│     │       ├── ID: toolu_6f265013-04f                      │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "user_id": "ENG004"                   │
│     │               }                                       │
│     └── Block 4                                             │
│         └── Tool Use: get_custom_budget                     │
│             ├── ID: toolu_c7a1e63d-6e6                      │
│             ├── Caller: code execution environment          │
│             └── Input:                                      │
│                 └── {                                       │
│                       "user_id": "ENG007"                   │
│                     }                                       │
╰─────────────────────────────────────────────────────────────╯


[Container] ID: container_55a4b5c74f93
[Container] Expires at: 2025-12-29 13:52:54.239663
[PTC] Tool called from code execution environment: get_custom_budget
[PTC] Tool called from code execution environment: get_custom_budget
[PTC] Tool called from code execution environment: get_custom_budget
[PTC] Tool called from code execution environment: get_custom_budget


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.1.content.0.type: Expected `thinking` or `redacted_thinking`, but found `text`. When `thinking` is enabled, a final `assistant` message must start with a thinking block (preceeding the lastmost set of `tool_use` and `tool_result` blocks). We recommend you include thinking blocks from previous turns. To avoid this requirement, disable `thinking`. Please consult our documentation at https://docs.claude.com/en/docs/build-with-claude/extended-thinking'}}